In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_all_only_2.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_all_only_2.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,CH,CH,136.0
1,US,US,108822.0
2,GB,GB,11283.0
3,PL,PL,65.0
4,ZA,ZA,1909.0
...,...,...,...
1073,TH,MY,1.0
1074,SK,CA,1.0
1075,CA,AR,1.0
1076,PL,DK,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

118405.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,136.0,,,,
1,US,US,108822.0,,,,
2,GB,GB,11283.0,,,,
3,PL,PL,65.0,,,,
4,ZA,ZA,1909.0,,,,
...,...,...,...,...,...,...,...
1073,TH,MY,1.0,,,,
1074,SK,CA,1.0,,,,
1075,CA,AR,1.0,,,,
1076,PL,DK,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,136.0,542.0,542.0,273.0,269.0
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0
3,PL,PL,65.0,231.0,231.0,104.0,127.0
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0
...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0
1076,PL,DK,1.0,231.0,201.0,104.0,118.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,136.0,542.0,542.0,273.0,269.0,,
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0,,
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0,,
3,PL,PL,65.0,231.0,231.0,104.0,127.0,,
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0,,
...,...,...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0,,
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0,,
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0,,
1076,PL,DK,1.0,231.0,201.0,104.0,118.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,136.0,542.0,542.0,273.0,269.0,50.557621,49.81685
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0,93.814495,91.906592
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0,74.637825,74.980064
3,PL,PL,65.0,231.0,231.0,104.0,127.0,51.181102,62.5
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0,81.337878,83.108402
...,...,...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0,0.724638,4.545455
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0,0.012238,4.166667
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0,7.142857,0.013732
1076,PL,DK,1.0,231.0,201.0,104.0,118.0,0.847458,0.961538


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,136.0,542.0,542.0,273.0,269.0,50.557621,49.81685,,,
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0,93.814495,91.906592,,,
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0,74.637825,74.980064,,,
3,PL,PL,65.0,231.0,231.0,104.0,127.0,51.181102,62.5,,,
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0,81.337878,83.108402,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0,0.724638,4.545455,,,
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0,0.012238,4.166667,,,
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0,7.142857,0.013732,,,
1076,PL,DK,1.0,231.0,201.0,104.0,118.0,0.847458,0.961538,,,


In [19]:
import pypopulation

In [20]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [21]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,136.0,542.0,542.0,273.0,269.0,50.557621,49.81685,0.000008,,
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0,93.814495,91.906592,0.000166,,
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0,74.637825,74.980064,0.000084,,
3,PL,PL,65.0,231.0,231.0,104.0,127.0,51.181102,62.5,0.000001,,
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0,81.337878,83.108402,0.000016,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0,0.724638,4.545455,0.0,,
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0,0.012238,4.166667,0.0,,
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0,7.142857,0.013732,0.0,,
1076,PL,DK,1.0,231.0,201.0,104.0,118.0,0.847458,0.961538,0.0,,


In [22]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [23]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,136.0,542.0,542.0,273.0,269.0,50.557621,49.81685,0.000008,0.125461,0.250923
1,US,US,108822.0,234402.0,234402.0,118405.0,115997.0,93.814495,91.906592,0.000166,0.232127,0.464254
2,GB,GB,11283.0,30165.0,30165.0,15048.0,15117.0,74.637825,74.980064,0.000084,0.187021,0.374043
3,PL,PL,65.0,231.0,231.0,104.0,127.0,51.181102,62.5,0.000001,0.140693,0.281385
4,ZA,ZA,1909.0,4644.0,4644.0,2297.0,2347.0,81.337878,83.108402,0.000016,0.205534,0.411068
...,...,...,...,...,...,...,...,...,...,...,...,...
1073,TH,MY,1.0,40.0,247.0,22.0,138.0,0.724638,4.545455,0.0,0.003484,0.00625
1074,SK,CA,1.0,56.0,15453.0,24.0,8171.0,0.012238,4.166667,0.0,0.000064,0.000122
1075,CA,AR,1.0,15453.0,26.0,7282.0,14.0,7.142857,0.013732,0.0,0.000065,0.000137
1076,PL,DK,1.0,231.0,201.0,104.0,118.0,0.847458,0.961538,0.0,0.002315,0.004505


In [24]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_all_only_2.csv', index=False)